In [78]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F

In [79]:
import sys

# Add the path to your module's directory
sys.path.append('/home/vladyslav_podrazhanskyi/projects/PERSONAL/python/learn_spark')

# Now you can import your 
from my_code.utils import ROOT

In [80]:
spark = SparkSession.builder.master("local[*]").getOrCreate()
print(spark)

In [81]:
myRange = spark.range(1000).toDF("number")
myRange.show()

+------+
|number|
+------+
|     0|
|     1|
|     2|
|     3|
|     4|
|     5|
|     6|
|     7|
|     8|
|     9|
|    10|
|    11|
|    12|
|    13|
|    14|
|    15|
|    16|
|    17|
|    18|
|    19|
+------+
only showing top 20 rows



In [82]:
divisBy2 = myRange.where("number % 2 = 0")

In [83]:
flightData2015 = spark \
    .read \
    .option("inferSchema", "true") \
    .option("header", "true") \
    .csv(f"{ROOT}/source_data/flight-data/csv/2015-summary.csv")

In [84]:
flightData2015.show()

+--------------------+-------------------+-----+
|   DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+--------------------+-------------------+-----+
|       United States|            Romania|   15|
|       United States|            Croatia|    1|
|       United States|            Ireland|  344|
|               Egypt|      United States|   15|
|       United States|              India|   62|
|       United States|          Singapore|    1|
|       United States|            Grenada|   62|
|          Costa Rica|      United States|  588|
|             Senegal|      United States|   40|
|             Moldova|      United States|    1|
|       United States|       Sint Maarten|  325|
|       United States|   Marshall Islands|   39|
|              Guyana|      United States|   64|
|               Malta|      United States|    1|
|            Anguilla|      United States|   41|
|             Bolivia|      United States|   30|
|       United States|           Paraguay|    6|
|             Algeri

In [85]:
print(type(flightData2015.take(3)[0]))   # <class 'pyspark.sql.types.Row'>

<class 'pyspark.sql.types.Row'>


In [86]:

flightData2015.take(3)
# [Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Romania', count=15),
# Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Croatia', count=1),
# Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Ireland', count=344)]
# """

[Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Romania', count=15),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Croatia', count=1),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Ireland', count=344)]

In [87]:
flightData2015.sort("count", ascending=False).show()

+------------------+-------------------+------+
| DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME| count|
+------------------+-------------------+------+
|     United States|      United States|370002|
|     United States|             Canada|  8483|
|            Canada|      United States|  8399|
|     United States|             Mexico|  7187|
|            Mexico|      United States|  7140|
|    United Kingdom|      United States|  2025|
|     United States|     United Kingdom|  1970|
|             Japan|      United States|  1548|
|     United States|              Japan|  1496|
|           Germany|      United States|  1468|
|     United States| Dominican Republic|  1420|
|Dominican Republic|      United States|  1353|
|     United States|            Germany|  1336|
|       South Korea|      United States|  1048|
|     United States|        The Bahamas|   986|
|       The Bahamas|      United States|   955|
|     United States|             France|   952|
|            France|      United States|

In [88]:
(
    flightData2015
    .where(F.col("DEST_COUNTRY_NAME") != 'United States')
    .sort("count", ascending=False)
    .show()
)

+------------------+-------------------+-----+
| DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+------------------+-------------------+-----+
|            Canada|      United States| 8399|
|            Mexico|      United States| 7140|
|    United Kingdom|      United States| 2025|
|             Japan|      United States| 1548|
|           Germany|      United States| 1468|
|Dominican Republic|      United States| 1353|
|       South Korea|      United States| 1048|
|       The Bahamas|      United States|  955|
|            France|      United States|  935|
|          Colombia|      United States|  873|
|            Brazil|      United States|  853|
|       Netherlands|      United States|  776|
|             China|      United States|  772|
|           Jamaica|      United States|  666|
|        Costa Rica|      United States|  588|
|       El Salvador|      United States|  561|
|            Panama|      United States|  510|
|              Cuba|      United States|  466|
|            

In [89]:
# You can make any DataFrame into a table or view with one simple method call:
flightData2015.createOrReplaceTempView("flight_data_2015")

In [90]:
sqlWay = spark.sql("""
SELECT DEST_COUNTRY_NAME, Sum(count) AS sum
FROM flight_data_2015
GROUP BY DEST_COUNTRY_NAME
ORDER BY sum DESC
""")

In [91]:
sqlWay.show()

+------------------+------+
| DEST_COUNTRY_NAME|   sum|
+------------------+------+
|     United States|411352|
|            Canada|  8399|
|            Mexico|  7140|
|    United Kingdom|  2025|
|             Japan|  1548|
|           Germany|  1468|
|Dominican Republic|  1353|
|       South Korea|  1048|
|       The Bahamas|   955|
|            France|   935|
|          Colombia|   873|
|            Brazil|   853|
|       Netherlands|   776|
|             China|   772|
|           Jamaica|   666|
|        Costa Rica|   588|
|       El Salvador|   561|
|            Panama|   510|
|              Cuba|   466|
|             Spain|   420|
+------------------+------+
only showing top 20 rows



In [92]:
dataFrameWay = flightData2015 \
    .groupBy("DEST_COUNTRY_NAME") \
    .sum("count") \
    .withColumnRenamed("sum(count)", "sum") \
    .orderBy("sum", ascending=False)

dataFrameWay.show()

+------------------+------+
| DEST_COUNTRY_NAME|   sum|
+------------------+------+
|     United States|411352|
|            Canada|  8399|
|            Mexico|  7140|
|    United Kingdom|  2025|
|             Japan|  1548|
|           Germany|  1468|
|Dominican Republic|  1353|
|       South Korea|  1048|
|       The Bahamas|   955|
|            France|   935|
|          Colombia|   873|
|            Brazil|   853|
|       Netherlands|   776|
|             China|   772|
|           Jamaica|   666|
|        Costa Rica|   588|
|       El Salvador|   561|
|            Panama|   510|
|              Cuba|   466|
|             Spain|   420|
+------------------+------+
only showing top 20 rows



In [93]:
(
    flightData2015
    .where(F.col('DEST_COUNTRY_NAME') != "Unites States")
    .explain()
)

== Physical Plan ==
*(1) Filter (isnotnull(DEST_COUNTRY_NAME#620) AND NOT (DEST_COUNTRY_NAME#620 = Unites States))
+- FileScan csv [DEST_COUNTRY_NAME#620,ORIGIN_COUNTRY_NAME#621,count#622] Batched: false, DataFilters: [isnotnull(DEST_COUNTRY_NAME#620), NOT (DEST_COUNTRY_NAME#620 = Unites States)], Format: CSV, Location: InMemoryFileIndex(1 paths)[file:/home/vladyslav_podrazhanskyi/projects/PERSONAL/python/learn_spar..., PartitionFilters: [], PushedFilters: [IsNotNull(DEST_COUNTRY_NAME), Not(EqualTo(DEST_COUNTRY_NAME,Unites States))], ReadSchema: struct<DEST_COUNTRY_NAME:string,ORIGIN_COUNTRY_NAME:string,count:int>




== Physical Plan ==

*(1) Filter (isnotnull(DEST_COUNTRY_NAME#20) AND NOT (DEST_COUNTRY_NAME#20 = Unites States))

FileScan csv [DEST_COUNTRY_NAME#20,ORIGIN_COUNTRY_NAME#21,count#22]
Batched: false,
DataFilters: [],
Format: CSV,
Location: InMemoryFileIndex(1 paths)[file:/home/vladyslav_podrazhanskyi/projects/PERSONAL/python/learn_spar...,
PartitionFilters: [],
PushedFilters: [],
ReadSchema: struct<DEST_COUNTRY_NAME:string,ORIGIN_COUNTRY_NAME:string,count:int>

In [94]:
(
    flightData2015
    .sort('count', ascending=False)
    .explain()
)

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- Sort [count#622 DESC NULLS LAST], true, 0
   +- Exchange rangepartitioning(count#622 DESC NULLS LAST, 200), ENSURE_REQUIREMENTS, [plan_id=1233]
      +- FileScan csv [DEST_COUNTRY_NAME#620,ORIGIN_COUNTRY_NAME#621,count#622] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex(1 paths)[file:/home/vladyslav_podrazhanskyi/projects/PERSONAL/python/learn_spar..., PartitionFilters: [], PushedFilters: [], ReadSchema: struct<DEST_COUNTRY_NAME:string,ORIGIN_COUNTRY_NAME:string,count:int>




== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- Sort [count#22 ASC NULLS FIRST], true, 0
   +- Exchange rangepartitioning(count#22 ASC NULLS FIRST, 200), ENSURE_REQUIREMENTS, [id=#124]
      +- FileScan csv[DEST_COUNTRY_NAME  # 20,ORIGIN_COUNTRY_NAME#21,count#22]....

In [95]:
sqlWay.explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- Sort [sum#681L DESC NULLS LAST], true, 0
   +- Exchange rangepartitioning(sum#681L DESC NULLS LAST, 200), ENSURE_REQUIREMENTS, [plan_id=1253]
      +- HashAggregate(keys=[DEST_COUNTRY_NAME#620], functions=[sum(count#622)])
         +- Exchange hashpartitioning(DEST_COUNTRY_NAME#620, 200), ENSURE_REQUIREMENTS, [plan_id=1250]
            +- HashAggregate(keys=[DEST_COUNTRY_NAME#620], functions=[partial_sum(count#622)])
               +- FileScan csv [DEST_COUNTRY_NAME#620,count#622] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex(1 paths)[file:/home/vladyslav_podrazhanskyi/projects/PERSONAL/python/learn_spar..., PartitionFilters: [], PushedFilters: [], ReadSchema: struct<DEST_COUNTRY_NAME:string,count:int>




== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- Sort [sum#77L DESC NULLS LAST], true, 0
   +- Exchange rangepartitioning(sum#77L DESC NULLS LAST, 200), ENSURE_REQUIREMENTS, [id=#144]
      +- HashAggregate(keys=[DEST_COUNTRY_NAME#20], functions=[sum(count#22)])
         +- Exchange hashpartitioning(DEST_COUNTRY_NAME#20, 200), ENSURE_REQUIREMENTS, [id=#141]
            +- HashAggregate(keys=[DEST_COUNTRY_NAME#20], functions=[partial_sum(count#22)])
               +- FileScan csv [DEST_COUNTRY_NAME#20,count#22] Batched: false, DataFil

In [96]:
dataFrameWay.explain()
# == Physical Plan ==
# AdaptiveSparkPlan isFinalPlan=false

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- Sort [sum#707L DESC NULLS LAST], true, 0
   +- Exchange rangepartitioning(sum#707L DESC NULLS LAST, 200), ENSURE_REQUIREMENTS, [plan_id=1273]
      +- HashAggregate(keys=[DEST_COUNTRY_NAME#620], functions=[sum(count#622)])
         +- Exchange hashpartitioning(DEST_COUNTRY_NAME#620, 200), ENSURE_REQUIREMENTS, [plan_id=1270]
            +- HashAggregate(keys=[DEST_COUNTRY_NAME#620], functions=[partial_sum(count#622)])
               +- FileScan csv [DEST_COUNTRY_NAME#620,count#622] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex(1 paths)[file:/home/vladyslav_podrazhanskyi/projects/PERSONAL/python/learn_spar..., PartitionFilters: [], PushedFilters: [], ReadSchema: struct<DEST_COUNTRY_NAME:string,count:int>




In [97]:
print(flightData2015.select(F.max("count")).take(1))  



[Row(max(count)=370002)]


In [98]:
maxSql = spark.sql("""
SELECT DEST_COUNTRY_NAME, sum(count) as destination_total
FROM flight_data_2015
GROUP BY DEST_COUNTRY_NAME
ORDER BY sum(count) DESC
LIMIT 5
""")

print('maxSql')
maxSql.show()



maxSql
+-----------------+-----------------+
|DEST_COUNTRY_NAME|destination_total|
+-----------------+-----------------+
|    United States|           411352|
|           Canada|             8399|
|           Mexico|             7140|
|   United Kingdom|             2025|
|            Japan|             1548|
+-----------------+-----------------+



In [99]:
flightData2015 \
    .groupBy("DEST_COUNTRY_NAME") \
    .sum("count") \
    .withColumnRenamed("sum(count)", "sum") \
    .sort(F.desc("sum")) \
    .limit(5) \
    .show()




+-----------------+------+
|DEST_COUNTRY_NAME|   sum|
+-----------------+------+
|    United States|411352|
|           Canada|  8399|
|           Mexico|  7140|
|   United Kingdom|  2025|
|            Japan|  1548|
+-----------------+------+



In [100]:
flightData2015.show()

+--------------------+-------------------+-----+
|   DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+--------------------+-------------------+-----+
|       United States|            Romania|   15|
|       United States|            Croatia|    1|
|       United States|            Ireland|  344|
|               Egypt|      United States|   15|
|       United States|              India|   62|
|       United States|          Singapore|    1|
|       United States|            Grenada|   62|
|          Costa Rica|      United States|  588|
|             Senegal|      United States|   40|
|             Moldova|      United States|    1|
|       United States|       Sint Maarten|  325|
|       United States|   Marshall Islands|   39|
|              Guyana|      United States|   64|
|               Malta|      United States|    1|
|            Anguilla|      United States|   41|
|             Bolivia|      United States|   30|
|       United States|           Paraguay|    6|
|             Algeri

In [101]:
spark.stop()
